# Week 1: It is starting.


In [ ]:
#retrieving friends lists

import csv
import re
import matplotlib.pyplot as plt

#initializing dictionnary of friends
#this dictionnary contains every user's list of friends using user IDs
friends_dict={}
with open('data/friends.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
      friends_dict[row['# Friendship between two user IDs is indicated by a comma-separated entry.']]=[]

#filling the dictionnary
with open('data/friends.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
      friends_dict[row['# Friendship between two user IDs is indicated by a comma-separated entry.']].append(row[None][0])

print('user 360\'s friends list: ',friends_dict['1612'])


#retrieving number of concerts

#this dictionnary contains the number of concerts for one genre
concert_numbers={}
with open('data/n_concerts.txt', 'r') as f:
    lines = f.readlines()
    for l in lines:
      x=re.search('([\w| |/|&|-]+):([0-9]+)',l)
      if x:
        concert_numbers[x.group(1)]=[x.group(2)][0]

print('number of classical concerts: ',concert_numbers['Classical'])

In [ ]:
# retrieving preferences

import json

with open('data/preferences.json', 'r') as f:
  preferences_data = json.load(f)

with open('data/preferences_description.txt') as f:
  pref_des_data_lines = f.readlines()
# get the genres a user likes using their ID
def getGenresFromId(id):
  genresNum=[]
  genres=[]
  
  preferences=preferences_data[str(id)]
  for i in range(len(preferences)):
    if preferences[i]=='1':
      genresNum.append(i)
  for num in genresNum:
    genres.append(pref_des_data_lines[num+2].rstrip())
  return(genres)
  
getGenresFromId(1612)

In [ ]:
# calculates the number of concert two friends (id1,id2) are attending together
def numberOfConcerts2Friends(id1,id2):
  total_number_of_concert=0
  # calculates the number of concert for every genre they both like
  genres_both_like= [element for element in getGenresFromId(id1) if element in getGenresFromId(id2)]
  for g in genres_both_like:
    total_number_of_concert+=0.393*float(concert_numbers[g])
  # calculates the number of concert for every genre only one of them likes
  genres_one_likes=list(set(getGenresFromId(id1)).symmetric_difference(getGenresFromId(id2)))
  for g in genres_one_likes:
    total_number_of_concert+=0.018*float(concert_numbers[g])
  genres=concert_numbers.keys()
  # calculates the number of concert for every genre none of them likes
  genres_none_like=[element for element in genres if element not in list(set(genres_both_like)|set(genres_one_likes))]
  for g in genres_none_like:
    total_number_of_concert+=0.002*float(concert_numbers[g])
  return total_number_of_concert
numberOfConcerts2Friends(360,4719)

In [ ]:
def numberOfConcerts1User(id):
  res=0
  for f in friends_dict[str(id)]:
    res+=numberOfConcerts2Friends(id,int(f))
  return res
  

In [ ]:
from collections import OrderedDict

def create_textfile_of_users():
    weights_of_users = OrderedDict({})
    # fill dict
    for user in friends_dict.keys():
        weights_of_users[user] = numberOfConcerts1User(int(user))

    # sorted dict of useres and number of concerts they go to
    weights_of_users_list = sorted(weights_of_users.items(), key=lambda x:x[1], reverse=True)

    # 12% threshold
    twelve_tresh = int(len(weights_of_users_list)*0.12)
    # create a string of users
    long_string = ""
    for counter in range (twelve_tresh):
        user = weights_of_users_list[counter]
        long_string += str(user[0])+"\n"

    # write into txt file
    with open('a_team_8.txt', 'w') as f:
        f.write(long_string)
    f.close()


In [ ]:
import networkx as nx
def create_network():
    # create an empty graph
    G = nx.Graph()

    # add nodes to the graph representing individual people
    for user in friends_dict.keys():
        G.add_node(user)
    
    # add edges to the graph representing co-visits to concerts
    for user in friends_dict.keys():
        for friend in friends_dict[user]:
            if not G.has_edge(user,friend):
                weight=numberOfConcerts2Friends(int(user),int(friend))
                G.add_edge(user, friend,weight=weight)
    print(G.number_of_edges())
    degree_centrality = nx.degree_centrality(G)

create_network()
